In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_excel("data/IPH_robos_2015_2020_tecmty (1).xlsx")
df2 = pd.read_excel("data/IPH_robos_2021_2024_tecmty_fechas_corregidas.xlsx")
df3 = pd.read_excel("data/IPH_robos_ene-ago-2025_tecmty.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
df3.isna().sum()

In [ ]:
# Del df3 cambiar el nombre de VOLENCIA por VIOLENCIA
df3 = df3.rename(columns={"VOLENCIA": "VIOLENCIA"})

In [ ]:
# Eliminar la columna "DIA"
df2 = df2.drop(columns=["dia"])

In [ ]:
# Juntar los dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)


In [ ]:
df.head()

In [ ]:
# Quiero saber cuántas filas hay en total
print(f"Total de filas en el dataframe combinado: {df.shape[0]}")

In [ ]:
df.dtypes

In [ ]:
df.duplicated(subset=['FOLIO']).sum()

In [ ]:
df.isna().sum()

In [ ]:
# Que queden las nuevas columnas de año, mes y día sin decimales
df['AÑO'] = pd.DatetimeIndex(df['FECHA']).year
df['MES'] = pd.DatetimeIndex(df['FECHA']).month
df['DIA'] = pd.DatetimeIndex(df['FECHA']).day
df.head()


In [ ]:
# Crear la columna NOMBRE_DIA con el nombre del día de la semana
df['NOMBRE_DIA'] = df['FECHA'].dt.day_name()
df.head()

In [ ]:
df.groupby(['DISTRITO', 'TIPO']).size().unstack(fill_value=0)

In [ ]:
cuadrante_counts = pd.DataFrame({'CUADRANTE': range(1, 79)})
cuadrante_counts.head()

In [ ]:
# Crear una columna nueva en cuadrante_counts con el conteo de robos por cuadrante llamada NUM_ROBOS
cuadrante_robos = df['CUADRANTE'].value_counts().reset_index()
cuadrante_robos.columns = ['CUADRANTE', 'NUM_ROBOS']

In [ ]:
cuadrante_counts = cuadrante_counts.merge(cuadrante_robos, on='CUADRANTE', how='left')


In [ ]:
cuadrante_counts

In [ ]:
# Elimina el decimal de NUM_ROBOS
cuadrante_counts['NUM_ROBOS'] = cuadrante_counts['NUM_ROBOS'].fillna(0).astype(int)

In [ ]:
cuadrante_counts

In [ ]:
# Crear tablas por tipo: cuadrante_vehiculo, cuadrante_casa, cuadrante_negocio

# Dataframe base con todos los cuadrantes (1..78)
cuadrantes = pd.DataFrame({'CUADRANTE': range(1, 79)})
# Función auxiliar que cuenta robos por cuadrante según patrón en la columna TIPO
def tipo_counts(regex_pattern):
    mask = df['TIPO'].fillna('').astype(str).str.lower().str.contains(regex_pattern, regex=True)
    counts = df.loc[mask, 'CUADRANTE'].value_counts().reset_index()
    counts.columns = ['CUADRANTE', 'NUM_ROBOS']
    # Asegurar CUADRANTE entero (si acaso viene como float/str)
    counts['CUADRANTE'] = counts['CUADRANTE'].astype(int)
    # Merge con la lista completa de cuadrantes y rellenar ceros
    return cuadrantes.merge(counts, on='CUADRANTE', how='left').assign(NUM_ROBOS=lambda x: x['NUM_ROBOS'].fillna(0).astype(int))

# Patrones aproximados; ajústelos si los valores en TIPO son diferentes
cuadrante_vehiculo = tipo_counts(r'veh|auto|moto')
cuadrante_casa     = tipo_counts(r'casa|dom|hogar|vivi')
cuadrante_negocio  = tipo_counts(r'negocio|comer|tiend|local')

# Resumen rápido
print('Total robos (vehículo):', cuadrante_vehiculo['NUM_ROBOS'].sum())
print('Total robos (casa):',     cuadrante_casa['NUM_ROBOS'].sum())
print('Total robos (negocio):',  cuadrante_negocio['NUM_ROBOS'].sum())



In [ ]:
cuadrante_negocio.head()

In [ ]:
poblacion_cuadrante = pd.read_excel("data/poblacion cuadrantes.xlsx")
unidades_economicas_cuadrante = pd.read_excel("data/unidades economicas cuadrantes.xlsx")

In [ ]:
# Cambia los nombres de poblacion_cuadrante a CUADRANTE y POBLACION
poblacion_cuadrante.columns = ['CUADRANTE', 'POBLACION']

In [ ]:
# unir cuadrantes con poblacion_cuadrantes por la columna CUADRANTE

cuadrante_negocio = pd.merge(cuadrante_negocio, poblacion_cuadrante, on='CUADRANTE', how='left')
cuadrante_negocio.head()

In [ ]:
cuadrante_vehiculo = pd.merge(cuadrante_vehiculo, poblacion_cuadrante, on='CUADRANTE', how='left')
cuadrante_vehiculo.head()

In [ ]:
cuadrante_casa = pd.merge(cuadrante_casa, poblacion_cuadrante, on='CUADRANTE', how='left')
cuadrante_casa.head()

In [ ]:
# De los 3 cuadrantes creados anteriormente, los NAN susituimos por 0 en la columna de POBLACION
cuadrante_negocio['POBLACION'].fillna(0, inplace=True)
cuadrante_vehiculo['POBLACION'].fillna(0, inplace=True)
cuadrante_casa['POBLACION'].fillna(0, inplace=True)

In [ ]:
cuadrante_negocio.head()

In [ ]:
# Quita el numero decimal en la columna de POBLACION
cuadrante_negocio['POBLACION'] = cuadrante_negocio['POBLACION'].astype(int)
cuadrante_vehiculo['POBLACION'] = cuadrante_vehiculo['POBLACION'].astype(int)
cuadrante_casa['POBLACION'] = cuadrante_casa['POBLACION'].astype(int)

In [ ]:
cuadrante_negocio.head()

In [ ]:
import re


# Filtrar por tipo (ajuste patrones si hace falta)
df_vehiculo = df[df['TIPO'].fillna('').astype(str).str.lower().str.contains(r'veh|auto|moto', regex=True)]
df_casa     = df[df['TIPO'].fillna('').astype(str).str.lower().str.contains(r'casa|dom|hogar|vivi', regex=True)]
df_negocio  = df[df['TIPO'].fillna('').astype(str).str.lower().str.contains(r'negocio|comer|tiend|local', regex=True)]

def build_cuadrante_table(source_df):
    src = source_df.copy()
    # asegurar CUADRANTE numérico e int (descartar filas sin cuadrante válido)
    src['CUADRANTE'] = pd.to_numeric(src.get('CUADRANTE'), errors='coerce')
    src = src.dropna(subset=['CUADRANTE']).copy()
    src['CUADRANTE'] = src['CUADRANTE'].astype(int)

    # Determinar columna/valor de violencia:
    # 1) si existe columna 'VIOLENCIA' usarla
    # 2) si no, intentar extraer 'SI'/'NO' desde la columna 'TIPO'
    if 'VIOLENCIA' in src.columns:
        viol = src['VIOLENCIA'].fillna('').astype(str)
    else:
        # extraer la primera aparición de 'si' o 'no' como palabra en TIPO (case-insensitive)
        viol = src['TIPO'].fillna('').astype(str).str.extract(r'(?i)\b(si|no)\b', expand=False).fillna('')

    viol = viol.str.strip().str.upper()  # normalizar a 'SI' / 'NO' / ''
    src = src.assign(VIOL=viol)

    # Agregar conteos por cuadrante
    agg = src.groupby('CUADRANTE').agg(
        NUM_ROBOS = ('VIOL', 'size'),
        CON_VIOLENCIA = ('VIOL', lambda x: (x == 'SI').sum()),
        SIN_VIOLENCIA = ('VIOL', lambda x: (x == 'NO').sum())
    ).reset_index()

    # Merge con la lista completa de cuadrantes y rellenar ceros
    res = cuadrantes.merge(agg, on='CUADRANTE', how='left').fillna(0)
    res['NUM_ROBOS'] = res['NUM_ROBOS'].astype(int)
    res['CON_VIOLENCIA'] = res['CON_VIOLENCIA'].astype(int)
    res['SIN_VIOLENCIA'] = res['SIN_VIOLENCIA'].astype(int)
    return res

# Crear tablas finales
cuadrante_vehiculo = build_cuadrante_table(df_vehiculo)
cuadrante_casa     = build_cuadrante_table(df_casa)
cuadrante_negocio  = build_cuadrante_table(df_negocio)

# Resumen rápido
print('Vehículo — robos totales:', cuadrante_vehiculo['NUM_ROBOS'].sum(),
      ' con violencia:', cuadrante_vehiculo['CON_VIOLENCIA'].sum(),
      ' sin violencia:', cuadrante_vehiculo['SIN_VIOLENCIA'].sum())
print('Casa     — robos totales:', cuadrante_casa['NUM_ROBOS'].sum(),
      ' con violencia:', cuadrante_casa['CON_VIOLENCIA'].sum(),
      ' sin violencia:', cuadrante_casa['SIN_VIOLENCIA'].sum())
print('Negocio  — robos totales:', cuadrante_negocio['NUM_ROBOS'].sum(),
      ' con violencia:', cuadrante_negocio['CON_VIOLENCIA'].sum(),
      ' sin violencia:', cuadrante_negocio['SIN_VIOLENCIA'].sum())


In [ ]:
cuadrante_negocio.head()

In [ ]:
def add_month_columns(cuadrante_df, source_df):
    # Pivot: cuenta por CUADRANTE y MES
    pivot = source_df.groupby(['CUADRANTE', 'MES']).size().unstack(fill_value=0)
    # Asegurar columnas de 1 a 12
    for m in range(1, 13):
        if m not in pivot.columns:
            pivot[m] = 0
    pivot = pivot[sorted(pivot.columns)]
    # Renombrar columnas a MES_1 ... MES_12
    pivot.columns = [f'MES_{int(c)}' for c in pivot.columns]
    pivot = pivot.reset_index()
    # Merge con el dataframe de cuadrantes
    res = pd.merge(cuadrante_df, pivot, on='CUADRANTE', how='left')
    # Rellenar NaN y casteo a int
    mes_cols = [f'MES_{i}' for i in range(1, 13)]
    for col in mes_cols:
        if col not in res.columns:
            res[col] = 0
        res[col].fillna(0, inplace=True)
        res[col] = res[col].astype(int)
    return res

cuadrante_negocio = add_month_columns(cuadrante_negocio, df_negocio)
cuadrante_casa = add_month_columns(cuadrante_casa, df_casa)
cuadrante_vehiculo = add_month_columns(cuadrante_vehiculo, df_vehiculo)

In [ ]:
cuadrante_negocio.head()

In [ ]:
# Agregar nueva columna de TOTAL_ROBOS que sea el total de los 3 tipos de robos por cuadrante en cada uno de los dataframes
# (asegurar que las tablas por tipo tengan columna NUM_ROBOS)
def _ensure_num(col_df):
    if 'NUM_ROBOS' not in col_df.columns:
        # buscar nombres alternativos y asignar a NUM_ROBOS si existen
        for alt in ['TOTAL_ROBOS_VEHICULO', 'TOTAL_ROBOS_CASA', 'TOTAL_ROBOS_NEGOCIO']:
            if alt in col_df.columns:
                col_df['NUM_ROBOS'] = col_df[alt]
                break
    return col_df

cuadrante_vehiculo = _ensure_num(cuadrante_vehiculo)
cuadrante_casa     = _ensure_num(cuadrante_casa)
cuadrante_negocio  = _ensure_num(cuadrante_negocio)

# Construir tabla con los tres conteos por cuadrante
totales = cuadrantes[['CUADRANTE']].copy()
totales = totales.merge(cuadrante_vehiculo[['CUADRANTE','NUM_ROBOS']].rename(columns={'NUM_ROBOS':'ROBOS_VEHICULO'}),
                        on='CUADRANTE', how='left')
totales = totales.merge(cuadrante_casa[['CUADRANTE','NUM_ROBOS']].rename(columns={'NUM_ROBOS':'ROBOS_CASA'}),
                        on='CUADRANTE', how='left')
totales = totales.merge(cuadrante_negocio[['CUADRANTE','NUM_ROBOS']].rename(columns={'NUM_ROBOS':'ROBOS_NEGOCIO'}),
                        on='CUADRANTE', how='left')

# Rellenar NAs y convertir a int
for col in ['ROBOS_VEHICULO','ROBOS_CASA','ROBOS_NEGOCIO']:
    totales[col] = totales[col].fillna(0).astype(int)

# Total combinado
totales['TOTAL_ROBOS_TODOS_TIPOS'] = totales[['ROBOS_VEHICULO','ROBOS_CASA','ROBOS_NEGOCIO']].sum(axis=1)

# Agregar la columna TOTAL_ROBOS a cada dataframe tipo (misma cifra por cuadrante)
cuadrante_vehiculo = cuadrante_vehiculo.merge(totales[['CUADRANTE','TOTAL_ROBOS_TODOS_TIPOS']], on='CUADRANTE', how='left')
cuadrante_casa     = cuadrante_casa.merge(totales[['CUADRANTE','TOTAL_ROBOS_TODOS_TIPOS']], on='CUADRANTE', how='left')
cuadrante_negocio  = cuadrante_negocio.merge(totales[['CUADRANTE','TOTAL_ROBOS_TODOS_TIPOS']], on='CUADRANTE', how='left')

# Normalizar nombre de la nueva columna en cada dataframe
cuadrante_vehiculo['TOTAL_ROBOS'] = cuadrante_vehiculo['TOTAL_ROBOS_TODOS_TIPOS'].fillna(0).astype(int)
cuadrante_casa['TOTAL_ROBOS']     = cuadrante_casa['TOTAL_ROBOS_TODOS_TIPOS'].fillna(0).astype(int)
cuadrante_negocio['TOTAL_ROBOS']  = cuadrante_negocio['TOTAL_ROBOS_TODOS_TIPOS'].fillna(0).astype(int)

# Eliminar columna auxiliar
cuadrante_vehiculo.drop(columns=['TOTAL_ROBOS_TODOS_TIPOS'], inplace=True, errors='ignore')
cuadrante_casa.drop(columns=['TOTAL_ROBOS_TODOS_TIPOS'], inplace=True, errors='ignore')
cuadrante_negocio.drop(columns=['TOTAL_ROBOS_TODOS_TIPOS'], inplace=True, errors='ignore')


In [ ]:
cuadrante_negocio.head()

In [ ]:
# Agregar datos de población en cada dataframe tipo
poblacion_cuadrante = pd.read_excel("data/poblacion cuadrantes.xlsx")
poblacion_cuadrante.columns = ['CUADRANTE', 'POBLACION'] 
cuadrante_vehiculo = cuadrante_vehiculo.merge(poblacion_cuadrante, on='CUADRANTE', how='left')
cuadrante_casa = cuadrante_casa.merge(poblacion_cuadrante, on='CUADRANTE', how='left')
cuadrante_negocio = cuadrante_negocio.merge(poblacion_cuadrante, on='CUADRANTE', how='left')

In [ ]:
cuadrante_negocio.head()

In [ ]:
# De los 3 cuadrantes creados anteriormente, los NAN susituimos por 0 en la columna de POBLACION
cuadrante_negocio['POBLACION'].fillna(0, inplace=True)
cuadrante_vehiculo['POBLACION'].fillna(0, inplace=True)
cuadrante_casa['POBLACION'].fillna(0, inplace=True)

In [ ]:
# Quita el numero decimal en la columna de POBLACION
cuadrante_negocio['POBLACION'] = cuadrante_negocio['POBLACION'].astype(int)
cuadrante_vehiculo['POBLACION'] = cuadrante_vehiculo['POBLACION'].astype(int)
cuadrante_casa['POBLACION'] = cuadrante_casa['POBLACION'].astype(int)

In [ ]:
cuadrante_negocio.head()

In [ ]:
# Calcular tasa de robos por cada 10,000 habitantes y guardarla en la columna '%_RB_1000_HAB'
# (se siguió la instrucción de nombre de columna aunque la tasa es por 10,000)
def _add_rb_per_10000(df):
    # evitar división por cero; si POBLACION==0 dejar 0
    df['RB_PER_10000_HAB'] = np.where(
        df.get('POBLACION', 0) > 0,
        (df['TOTAL_ROBOS'] / df['POBLACION'] * 10000).round(2),
        0.0
    )
    return df

cuadrante_vehiculo = _add_rb_per_10000(cuadrante_vehiculo)
cuadrante_casa     = _add_rb_per_10000(cuadrante_casa)
cuadrante_negocio  = _add_rb_per_10000(cuadrante_negocio)


In [ ]:
cuadrante_negocio.head()

In [ ]:
# Calcular RIESGO_RELATIVO: combina tasa (%_RB_1000_HAB), tendencia reciente (últimos 180d vs 180-360d) y violencia
def compute_riesgo_relativo(cuadrante_df, source_df, recent_days=180, weight_tasa=0.5, weight_trend=0.3, weight_viol=0.2):
    src = source_df.copy()
    src['FECHA'] = pd.to_datetime(src['FECHA'], errors='coerce')
    max_date = src['FECHA'].max()
    if pd.isna(max_date):
        # sin fechas válidas, asignar ceros
        cuadrante_df['TREND_180D'] = 0
        cuadrante_df['TREND_NORM'] = 0.0
        cuadrante_df['%_VIOLENCIA'] = 0.0
        cuadrante_df['RIESGO_RELATIVO'] = 0.0
        return cuadrante_df

    recent_start = max_date - pd.Timedelta(days=recent_days)
    prev_start = max_date - pd.Timedelta(days=2*recent_days)
    prev_end = recent_start - pd.Timedelta(days=1)

    # Conteos por periodo y cuadrante
    recent_counts = src.loc[src['FECHA'].between(recent_start, max_date)].groupby('CUADRANTE').size().rename('RECENTE')
    prev_counts = src.loc[src['FECHA'].between(prev_start, prev_end)].groupby('CUADRANTE').size().rename('PREV')

    trend_df = pd.DataFrame({'CUADRANTE': cuadrante_df['CUADRANTE']}).set_index('CUADRANTE')
    trend_df = trend_df.join(recent_counts).join(prev_counts)
    trend_df['RECENTE'] = trend_df['RECENTE'].fillna(0).astype(int)
    trend_df['PREV'] = trend_df['PREV'].fillna(0).astype(int)

    # Métrica de tendencia: (RECENTE - PREV) / (PREV + 1) para evitar division por cero
    trend_df['TREND_180D'] = (trend_df['RECENTE'] - trend_df['PREV']) / (trend_df['PREV'] + 1)
    # Normalizar tendencia 0..1 por min-max (si todos iguales, dejar 0)
    tmin, tmax = trend_df['TREND_180D'].min(), trend_df['TREND_180D'].max()
    if tmax > tmin:
        trend_df['TREND_NORM'] = (trend_df['TREND_180D'] - tmin) / (tmax - tmin)
    else:
        trend_df['TREND_NORM'] = 0.0

    # Violencia: porcentaje de robos con violencia en el dataframe de cuadrantes (usar columnas CON_VIOLENCIA y NUM_ROBOS)
    # Si no existen, intentar inferir a partir de datos origen (pero aquí usamos lo ya agregado)
    if 'CON_VIOLENCIA' in cuadrante_df.columns and 'NUM_ROBOS' in cuadrante_df.columns:
        viol_pct = cuadrante_df.apply(lambda r: (r['CON_VIOLENCIA'] / r['NUM_ROBOS']) if r['NUM_ROBOS'] > 0 else 0.0, axis=1)
    else:
        # fallback: 0 si no hay info
        viol_pct = pd.Series(0.0, index=cuadrante_df.index)

    # Tasa: usar columna 'RB_PER_10000_HAB' si existe (ya es escala relativa); normalizar por min-max
    tasa_col = 'RB_PER_10000_HAB'
    if tasa_col in cuadrante_df.columns:
        tasa_vals = cuadrante_df[tasa_col].astype(float).fillna(0.0)
    else:
        tasa_vals = pd.Series(0.0, index=cuadrante_df.index)

    # Normalizar tasa 0..1
    smin, smax = tasa_vals.min(), tasa_vals.max()
    if smax > smin:
        tasa_norm = (tasa_vals - smin) / (smax - smin)
    else:
        tasa_norm = pd.Series(0.0, index=cuadrante_df.index)

    # Combinar con pesos y escalar a 0-100
    trend_norm = trend_df['TREND_NORM'].reindex(cuadrante_df['CUADRANTE']).reset_index(drop=True)
    tasa_norm = tasa_norm.reset_index(drop=True)
    viol_pct = viol_pct.reset_index(drop=True)

    riesgo_raw = (weight_tasa * tasa_norm) + (weight_trend * trend_norm) + (weight_viol * viol_pct)
    riesgo_score = (riesgo_raw.fillna(0) * 100).round(2)

    # Asignar columnas al dataframe de cuadrante
    cuadrante_df = cuadrante_df.reset_index(drop=True)
    cuadrante_df['TREND_180D'] = trend_df['TREND_180D'].reindex(cuadrante_df['CUADRANTE']).values
    cuadrante_df['TREND_NORM'] = trend_df['TREND_NORM'].reindex(cuadrante_df['CUADRANTE']).values
    cuadrante_df['%_VIOLENCIA'] = (viol_pct * 100).round(2)
    cuadrante_df['RIESGO_RELATIVO'] = riesgo_score

    return cuadrante_df

# Aplicar a los tres dataframes (usar los dataframes fuente con FECHA: df_vehiculo, df_casa, df_negocio)
cuadrante_vehiculo = compute_riesgo_relativo(cuadrante_vehiculo, df_vehiculo)
cuadrante_casa     = compute_riesgo_relativo(cuadrante_casa, df_casa)
cuadrante_negocio  = compute_riesgo_relativo(cuadrante_negocio, df_negocio)


In [ ]:
cuadrante_vehiculo.head()